In [1]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

## Initialization

### Imports

In [2]:
import os
import sys
import json
import torch
import warnings
import numpy as np
import pandas as pd
import seaborn as sns

from tqdm.notebook import tqdm
from matplotlib import pyplot as plt

sys.path.append("../code/")
# warnings.simplefilter("ignore", UserWarning)

In [3]:
from params import *

from inference.main_test import k_fold_inf_test

from data.dataset import InferenceDataset
from data.transforms import HE_preprocess

### Inference

In [4]:
# log_folder = "../logs/2021-03-18/0/"  # b5
# log_folder = "../logs/2021-03-26/3/"  # b5 512
# log_folder = "../logs/2021-03-27/1/"  # seresnext
# log_folder = "../logs/2021-03-28/1/"  # b6

# log_folder = "../logs/2021-03-29/7/"  # b5 + 10 ep
# log_folder = "../logs/2021-03-30/0/"  # b5 + 20 ep

# log_folder = "../logs/2021-03-31/0/"  # b4 512
# log_folder = "../logs/2021-04-01/2/"  # bot unext
# log_folder = "../logs/2021-04-02/3/"  # b6
# log_folder = "../logs/2021-04-04/1/"  # b0
log_folder = "../logs/2021-04-05/4/"  # b1
# log_folder = "../logs/2021-04-06/2/"  # b1 512
# log_folder = "../logs/2021-04-08/2/"  # b1 lovasz
# log_folder = "../logs/2021-04-08/5/"  # b2 fp16
# log_folder = "../logs/2021-04-09/0/"  # b2 fp32 (tbc)
# log_folder = "../logs/2021-04-09/12/"  # ++b1
# log_folder = "../logs/2021-04-10/1/"  # b1 fp16
# log_folder = "../logs/2021-04-10/3/"  # b1 fp32 (repro)
# log_folder = "../logs/2021-04-10/4/"  # b2 fp32

# log_folder = "../logs/2021-04-11/3/"  # b1 2 512

In [5]:
class Config:
    def __init__(self, **entries):
        self.__dict__.update(entries)

config = json.load(open(log_folder + 'config.json', 'r'))
config = Config(**config)

In [6]:
# Fields not defined in old configs :
try:
    _ = config.tile_size
except:
    config.tile_size = 256
    config.reduce_factor = 4
    
try:
    _ = config.use_bot
except:
    config.use_bot = False
    config.use_fpn = False
    
try:
    _ = config.double_model
except:
    config.double_model = False

In [7]:
df = pd.read_csv(DATA_PATH + "sample_submission.csv")

In [8]:
config.overlap_factor = 1.5
use_tta = False
global_threshold = 0.4
config.selected_folds = [0]
save = True

In [9]:
%%time
scores = k_fold_inf_test(
    config,
    df,
    log_folder=log_folder,
    use_full_size=False,
    global_threshold=global_threshold,
    use_tta=use_tta,
    save=save,
)


-------------   Fold 1 / 5  -------------


 -> Loading weights from ../logs/2021-04-05/4/Unet_efficientnet-b1_0.pt

 - Image 2ec3f1bb9
 - Image 3589adb90
 - Image d488c759a
 - Image aa05346ff
 - Image 57512b7f1
CPU times: user 1min 10s, sys: 10.2 s, total: 1min 20s
Wall time: 37.8 s
